<h1 align=center><font size = 6>Segmenting and Clustering</font></h1>

<h1 align=center><font size = 12>Neighborhoods in Toronto</font></h1>


# Table of Contents


### Question 1: 

#### 1.1. Create a new Notebook 

#### 1.2. Scrape Web Page

#### 1.3. Transform Data into Pandas Dataframe 

#### 1.4. Clean Dataframe and Annotate Notebook

### 1.5. Link Notebook on Github repository. (10 marks)

https://github.com/lindangulopez/Coursera_Capstone/blob/master/Peer-graded%20Assignment%20Q1_Toronto_Linda.ipynb


### Question 2: 

#### 2.1. Get the latitude and the longitude coordinates of each neighborhood, used Geopy:

https://geopy.readthedocs.io/en/stable/#installation

#### 2.2. Use Geocoder Python package: https://geocoder.readthedocs.io/index.html. 

#### 2.3. Use Geopy & OpenStreetMap to create Dataframe

#### 2.4. Retrieved only 5 coordinates with with lambda equation, 

#### did not create a loop for each postal code to remake the calls.

#### 2.5. Used link to a csv file, http://cocl.us/Geospatial_data to create Dataframe:

https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html 

### 2.6. Submit a link to your Notebook on your Github repository. (2 marks)

https://github.com/lindangulopez/Coursera_Capstone/blob/master/Q_2latlong.ipynb



### Question 3:

#### 3.1. Build a test set with boroughs in Toronto, see:
https://medium.com/@bobhaffner/folium-markerclusters-and-fastmarkerclusters-1e03b01cb7b1

#### 3.2. Replicate the same analysis with the full data set. 

#### 3.3. Explain what you decided to do and report observations.

#### 3.4. Generate maps to visualize your neighborhoods and how they cluster together.

#### 3.5. Submit a link to your Notebook on your Github repository. (3 marks)





<h1 align=center><font size = 8>...</font></h1>



# Question 1:


## 1.1.  Create a new Notebook 

with the basic dependencies.

In [87]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import requests # Library for web scraping

print('Libraries imported.')

Libraries imported.


## 1.2. Scrape Web Page

About the Data, Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, 
- is a list of postal codes in Canada where the first letter is M. Postal codes beginning with M are located within the city of Toronto in the province of Ontario. 

- Scraping table from HTML using BeautifulSoup, write a Python program similar to scrape.py,from:

### Corey Schafer Python Programming Tutorial:
The code from this video can be found at: https://github.com/CoreyMSchafer/code...




In [88]:
# To run this, you can install BeautifulSoup
# https://pypi.python.org/pypi/beautifulsoup4

# Or download the file
# http://beautiful-soup-4
# and unzip it in the same directory as this file
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
import ssl
import csv

print('BeautifulSoup  & csv imported.')

BeautifulSoup  & csv imported.


In [89]:
# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

print('SSL certificate errors ignored.')

SSL certificate errors ignored.


In [90]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(source, 'lxml')

#print(soup.prettify())
print('soup ready')

soup ready


In [91]:
table = soup.find('table',{'class':'wikitable sortable'})
#table

In [92]:

table_rows = table.find_all('tr')

#table_rows

In [93]:
data = []
for row in table_rows:
    data.append([t.text.strip() for t in row.find_all('td')])

df = pd.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighbourhood'])
df = df[~df['PostalCode'].isnull()]  # to filter out bad rows

#print(df.head(5))
#print('***')
#print(df.tail(5))

## 1.3. Transform Data into Pandas Dataframe 

In [94]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 288 entries, 1 to 288
Data columns (total 3 columns):
PostalCode       288 non-null object
Borough          288 non-null object
Neighbourhood    288 non-null object
dtypes: object(3)
memory usage: 9.0+ KB


In [95]:
df.shape

(288, 3)

## 1.4. Clean Dataframe and Annotate Notebook

Only process the cells that have an assigned borough, we can ignore cells with 'Not assigned' boroughs, like in rows 1 & 2.

In [96]:
import pandas
import requests
from bs4 import BeautifulSoup
website_text = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(website_text,'lxml')

table = soup.find('table',{'class':'wikitable sortable'})
table_rows = table.find_all('tr')

data = []
for row in table_rows:
    data.append([t.text.strip() for t in row.find_all('td')])

df = pandas.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighbourhood'])
df = df[~df['PostalCode'].isnull()]  # to filter out bad rows

#df.head(15)

In [97]:

df.drop(df[df['Borough']=="Not assigned"].index,axis=0, inplace=True)
#df.head()

The dataframe can be reindex as follows:

In [98]:
df1 = df.reset_index()
#df1.head()

In [99]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 211 entries, 0 to 210
Data columns (total 4 columns):
index            211 non-null int64
PostalCode       211 non-null object
Borough          211 non-null object
Neighbourhood    211 non-null object
dtypes: int64(1), object(3)
memory usage: 6.7+ KB


In [100]:
df1.shape

(211, 4)

More than one neighborhood can exist in one postal code area, M5A is listed twice and has two neighborhoods Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma using groupby, see: 

https://pandas-docs.github.io/pandas-docs-travis/user_guide/groupby.html


In [101]:
df2= df1.groupby('PostalCode').agg(lambda x: ','.join(x))

#df2.head()
       

In [102]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 103 entries, M1B to M9W
Data columns (total 2 columns):
Borough          103 non-null object
Neighbourhood    103 non-null object
dtypes: object(2)
memory usage: 2.4+ KB


In [103]:
df2.shape

(103, 2)

There are also cells that have an assigned neighbouhoods,like M7A, lets assign their boroughs as their neighbourhood, as follows:

In [104]:

df2.loc[df2['Neighbourhood']=="Not assigned",'Neighbourhood']=df2.loc[df2['Neighbourhood']=="Not assigned",'Borough']

#df2.head()



In [105]:
df3 = df2.reset_index()
#df3.head()

Now we can remove the duplicate boroughts as follows:

In [106]:
df3['Borough']= df3['Borough'].str.replace('nan|[{}\s]','').str.split(',').apply(set).str.join(',').str.strip(',').str.replace(",{2,}",",")

In [107]:
df3.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [108]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 3 columns):
PostalCode       103 non-null object
Borough          103 non-null object
Neighbourhood    103 non-null object
dtypes: object(3)
memory usage: 2.5+ KB


In [109]:
df3.shape

(103, 3)

## 1.5 Dataframe cleaned & shared on Github:



https://github.com/lindangulopez/Coursera_Capstone/blob/master/Peer-graded%20Assignment%20Q1_Toronto_Linda.ipynb


# Question 2: 



## 2.1. Get the latitude and the longitude coordinates of each neighborhood, used Geopy:

https://geopy.readthedocs.io/en/stable/#installation


In [110]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [111]:
from  geopy.geocoders import Nominatim
geolocator = Nominatim()
city ="London"
country ="Uk"
loc = geolocator.geocode(city+','+ country)
print("latitude is :-" ,loc.latitude,"\nlongtitude is:-" ,loc.longitude)

latitude is :- 51.5073219 
longtitude is:- -0.1276474


In [112]:
from  geopy.geocoders import Nominatim
geolocator = Nominatim()
location = geolocator.geocode("Toronto, North York, Parkwoods")

print(location.address)
print('')
print((location.latitude, location.longitude))
print('')
print(location.raw)


Parkwoods Village Drive, Parkway East, Don Valley East, North York, Toronto, Ontario, M3A 1Z5, Canada

(43.7612239, -79.3239857)

{'place_id': 113868274, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'way', 'osm_id': 160406963, 'boundingbox': ['43.761106', '43.761307', '-79.3242157', '-79.3239088'], 'lat': '43.7612239', 'lon': '-79.3239857', 'display_name': 'Parkwoods Village Drive, Parkway East, Don Valley East, North York, Toronto, Ontario, M3A 1Z5, Canada', 'class': 'highway', 'type': 'secondary', 'importance': 0.51}


In [113]:
import pandas as pd
#df3.head()

In [114]:

import pandas as pd
df_geopy = pd.DataFrame({'PostalCode': ['M3A', 'M4A', 'M5A'],
                         'Borough': ['North York', 'North York', 'Downtown Toronto'],
                         'Neighbourhood': ['Parkwoods', 'Victoria Village', 'Harbourfront'],})

from geopy.geocoders import Nominatim
geolocator = Nominatim()



In [115]:
df_geopy1 = df3
#df_geopy1

In [116]:
from geopy.geocoders import Nominatim
geolocator = Nominatim()

df_geopy1['address'] = df3[['PostalCode', 'Borough', 'Neighbourhood']].apply(lambda x: ', '.join(x), axis=1 )
df_geopy1.head()

,PostalCode,Borough,Neighbourhood,address
0,M1B,Scarborough,"Rouge,Malvern","M1B, Scarborough, Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union","M1C, Scarborough, Highland Creek,Rouge Hill,Po..."
2,M1E,Scarborough,"Guildwood,Morningside,West Hill","M1E, Scarborough, Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn,"M1G, Scarborough, Woburn"
4,M1H,Scarborough,Cedarbrae,"M1H, Scarborough, Cedarbrae"


In [117]:
df_geopy1 = df3

In [118]:
df_geopy1.shape


(103, 4)

In [119]:
df_geopy1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 4 columns):
PostalCode       103 non-null object
Borough          103 non-null object
Neighbourhood    103 non-null object
address          103 non-null object
dtypes: object(4)
memory usage: 3.3+ KB


In [120]:
df_geopy1.drop(df_geopy1[df_geopy1['Borough']=="Notassigned"].index,axis=0, inplace=True)
#df_geopy1
# code holds true up until i=102
df_geopy1.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 103 entries, 0 to 102
Data columns (total 4 columns):
PostalCode       103 non-null object
Borough          103 non-null object
Neighbourhood    103 non-null object
address          103 non-null object
dtypes: object(4)
memory usage: 4.0+ KB


In [121]:
#df_geopy1.head()


In [122]:
df_geopy1.shape


(103, 4)

In [123]:
df_geopy1.to_csv('geopy1.csv')
# no data for location after row 75

Now let's test for location = 'M1G, Scarborough, Woburn'

In [124]:
from  geopy.geocoders import Nominatim
geolocator = Nominatim()
location = geolocator.geocode("M1G, Scarborough, Woburn")


#print(location.address)

#print((location.latitude, location.longitude))

#print(location.raw)


### Great getting latitude & longitude, with geopy



## 2.2. Use Geocoder Python package: https://geocoder.readthedocs.io/index.html. 


In [125]:
pip install geocoder

Note: you may need to restart the kernel to use updated packages.



## 2.3. Use Geopy & OpenStreetMap to create Dataframe


In [126]:
df3.to_csv('geopy.csv')

In [127]:
import csv

with open('geopy.csv') as csvfile:
     reader = csv.DictReader(csvfile)
     #for row in reader:
         #print(row['PostalCode'],row['Borough'], row['Neighbourhood'] )

In [128]:
from  geopy.geocoders import Nominatim
geolocator = Nominatim()
location = geolocator.geocode("M1B Scarborough Rouge,Malvern")

#print(location.address)

#print((location.latitude, location.longitude))

#print(location.raw)


In [129]:
from  geopy.geocoders import Nominatim
geolocator = Nominatim()
location = geolocator.geocode("Toronto, Highland Creek")

#print(location.address)

#print((location.latitude, location.longitude))

#print(location.raw)

#M1C Scarborough Highland Creek,Rouge Hill,Port Union = no address

In [130]:
from  geopy.geocoders import Nominatim
geolocator = Nominatim()
location = geolocator.geocode("Toronto, Morningside")

#print(location.address)

#print((location.latitude, location.longitude))

#print(location.raw)

#M1E Scarborough Guildwood,Morningside,West Hill = no address


### Create csv file. 

In [131]:
import numpy as np
import csv


PostalCode = None
Borough = None
Neighbourhood = None
latData = None
longData = None

LAT_Woburn = 43.7598243
LONG_Woburn = -79.2252908
LAT_Malvern = 43.8091955
LONG_Malvern = -79.2217008
LAT_Highland_Creek = 43.7901172
LONG_Highland_Creek = -79.1733344
LAT_Morningside = 43.7826012
LONG_Morningside = -79.2049579

 
PostalCode = np.array(['M1H','M1B','M1C','M1G '])
Borough = np.array(['Scarborough','Scarborough','Scarborough','Scarborough'])
Neighbourhood = np.array(['Woburn','Malvern','Highland_Creek','Morningside'])
latData = np.array([43.7598243,43.8091955, 43.7901172 , 43.7826012])
longData = np.array([-79.2252908,-79.2217008,-79.1733344, -79.2049579 ])

with open('data.csv', 'w') as file:
    writer = csv.writer(file, delimiter=',')
    writer.writerow('ABXYZ')
    for a,b,x,y,z in np.nditer([ PostalCode.T, Borough.T, Neighbourhood.T, latData.T, longData.T], order='C'):
        writer.writerow([a,b,x,y,z])    

In [132]:
import csv

with open('data.csv') as csvfile:
     reader = csv.DictReader(csvfile)
     for row in reader:
         print(row['A'], row['B'],row['X'], row['Y'], row['Z'])

M1H Scarborough Woburn 43.7598243 -79.2252908
M1B Scarborough Malvern 43.8091955 -79.2217008
M1C Scarborough Highland_Creek 43.7901172 -79.1733344
M1G  Scarborough Morningside 43.7826012 -79.2049579


In [133]:
pd.read_csv('data.csv') 

,A,B,X,Y,Z
0,M1H,Scarborough,Woburn,43.759824,-79.225291
1,M1B,Scarborough,Malvern,43.809196,-79.221701
2,M1C,Scarborough,Highland_Creek,43.790117,-79.173334
3,M1G,Scarborough,Morningside,43.782601,-79.204958



## 2.4. Retrieve coordinates with with lambda equation, 

#### did not create a loop for each postal code to remake the calls.



### Read the docs:

https://docs.python.org/3.5/library/fileformats.html

https://geopy.readthedocs.io/en/latest/index.html#geopy.geocoders.Nominatim

#### Watch: 44 Python Tutorial Example Geocoding Addresses with Pandas and Geopy

https://www.youtube.com/watch?v=q_OUHA_zqeM



### Load DataFrame

In [134]:
import pandas, os
#os.listdir()

In [135]:
df_geopy=df3
#df_geopy.head()

In [136]:
import geopy
#dir(geopy)

In [137]:
type(df_geopy)

pandas.core.frame.DataFrame

In [138]:
df_geopy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 103 entries, 0 to 102
Data columns (total 4 columns):
PostalCode       103 non-null object
Borough          103 non-null object
Neighbourhood    103 non-null object
address          103 non-null object
dtypes: object(4)
memory usage: 4.0+ KB


### Import GeoPy:

In [139]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [140]:
from geopy.geocoders import Nominatim
print('Nominatim imported')

Nominatim imported


### Set connection to OpenStreeMap 

In [141]:
df_geopy['address']=df_geopy['PostalCode'] + ',' + df_geopy['Borough'] + ','+ df_geopy['Neighbourhood']
df_geopy.head()

,PostalCode,Borough,Neighbourhood,address
0,M1B,Scarborough,"Rouge,Malvern","M1B,Scarborough,Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union","M1C,Scarborough,Highland Creek,Rouge Hill,Port..."
2,M1E,Scarborough,"Guildwood,Morningside,West Hill","M1E,Scarborough,Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn,"M1G,Scarborough,Woburn"
4,M1H,Scarborough,Cedarbrae,"M1H,Scarborough,Cedarbrae"


In [142]:
nom = Nominatim()

In [143]:
n=nom.geocode('M1B, Scarborough, Rouge,Malvern')
n

Location(Malvern, Scarborough—Rouge Park, Scarborough, Toronto, Golden Horseshoe, Ontario, M1B 4Y7, Canada, (43.8091955, -79.2217008, 0.0))

In [144]:
n.latitude

43.8091955

In [145]:
type(n)

geopy.location.Location

### Watch out for None values

In [146]:
n2=nom.geocode('M1E Scarborough Guildwood,Morningside,West Hill')
print(n2)

None


In [147]:
n3=nom.geocode("Toronto, Morningside")
#n3

#### As you can see the postalcodes are different!

In [148]:
n4=nom.geocode("Toronto, Morningside, M1E ")
#print(n4)

In [149]:
n5=nom.geocode("Toronto, M1E ")
#print(n5)

#### None again, ...

### Use 'address' to get geocode coordinates:

Geocoding (Latitude/Longitude Lookup) Required parameters in a geocoding request: address — The street address that you want to geocode, in the format used by the national postal service of the country concerned. Additional address elements such as business names and unit, suite or floor numbers should be avoided.

In [150]:
df_geopy['Coordinates'] =df_geopy['address'].apply(nom.geocode)
df_geopy.head()



,PostalCode,Borough,Neighbourhood,address,Coordinates
0,M1B,Scarborough,"Rouge,Malvern","M1B,Scarborough,Rouge,Malvern","(Malvern, Scarborough—Rouge Park, Scarborough,..."
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union","M1C,Scarborough,Highland Creek,Rouge Hill,Port...",None
2,M1E,Scarborough,"Guildwood,Morningside,West Hill","M1E,Scarborough,Guildwood,Morningside,West Hill",None
3,M1G,Scarborough,Woburn,"M1G,Scarborough,Woburn","(Woburn, Scarborough—Guildwood, Scarborough, T..."
4,M1H,Scarborough,Cedarbrae,"M1H,Scarborough,Cedarbrae",None


### location object created at 'Coordinates', but there are only Coordinates 5 non-null objects! 


In [151]:
df_geopy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 103 entries, 0 to 102
Data columns (total 5 columns):
PostalCode       103 non-null object
Borough          103 non-null object
Neighbourhood    103 non-null object
address          103 non-null object
Coordinates      5 non-null object
dtypes: object(5)
memory usage: 4.8+ KB


In [152]:
df_geopy.Coordinates[0]

Location(Malvern, Scarborough—Rouge Park, Scarborough, Toronto, Golden Horseshoe, Ontario, M1B 4Y7, Canada, (43.8091955, -79.2217008, 0.0))

In [153]:
print(df_geopy.Coordinates[1])

None


In [154]:
df_geopy['latitude']=df_geopy['Coordinates'].apply(lambda x: x.latitude if x !=None else None)
df_geopy['longitude']=df_geopy['Coordinates'].apply(lambda x: x.longitude if x !=None else None)
df_geopy.head()

,PostalCode,Borough,Neighbourhood,address,Coordinates,latitude,longitude
0,M1B,Scarborough,"Rouge,Malvern","M1B,Scarborough,Rouge,Malvern","(Malvern, Scarborough—Rouge Park, Scarborough,...",43.809196,-79.221701
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union","M1C,Scarborough,Highland Creek,Rouge Hill,Port...",None,NaN,NaN
2,M1E,Scarborough,"Guildwood,Morningside,West Hill","M1E,Scarborough,Guildwood,Morningside,West Hill",None,NaN,NaN
3,M1G,Scarborough,Woburn,"M1G,Scarborough,Woburn","(Woburn, Scarborough—Guildwood, Scarborough, T...",43.759824,-79.225291
4,M1H,Scarborough,Cedarbrae,"M1H,Scarborough,Cedarbrae",None,NaN,NaN


In [155]:
df_geopy.to_csv('geo_loc_py.csv')

## As just 5 addresses were fruitful, we will go on to use the given geo-location data.

In [156]:
print('The latitude of', df_geopy.address[0],  'is', df_geopy.latitude[0], 'and its longitude is',df_geopy.longitude[0])

The latitude of M1B,Scarborough,Rouge,Malvern is 43.8091955 and its longitude is -79.2217008


#### Now let's merge the given dataframes:

In [157]:
# Load the Pandas libraries with alias 'pd' 
import pandas as pd 
# Read data from file 'filename.csv' 
# (in the same directory that your python process is based)
# Control delimiters, rows, column names with read_csv (see later) 
data2 = pd.read_csv("geopy.csv") 
# Preview the first 5 lines of the loaded data 
data2.head()

,Unnamed: 0,PostalCode,Borough,Neighbourhood,address
0,0,M1B,Scarborough,"Rouge,Malvern","M1B, Scarborough, Rouge,Malvern"
1,1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union","M1C, Scarborough, Highland Creek,Rouge Hill,Po..."
2,2,M1E,Scarborough,"Guildwood,Morningside,West Hill","M1E, Scarborough, Guildwood,Morningside,West Hill"
3,3,M1G,Scarborough,Woburn,"M1G, Scarborough, Woburn"
4,4,M1H,Scarborough,Cedarbrae,"M1H, Scarborough, Cedarbrae"


In [158]:
data3 = pd.read_csv("Geospatial_Coordinates.csv") 
# Preview the first 5 lines of the loaded data 
data3.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


- Rename 'Postal Code'

In [159]:
data3.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)
#data3.head()

In [160]:
data1 = pd.merge(data3, data2, how='inner', on=None, left_on=None, right_on=None,
         left_index=False, right_index=False, sort=True,
         suffixes=('_x', '_y'), copy=True, indicator=False,
         validate=None)

data1.head()

,PostalCode,Latitude,Longitude,Unnamed: 0,Borough,Neighbourhood,address
0,M1B,43.806686,-79.194353,0,Scarborough,"Rouge,Malvern","M1B, Scarborough, Rouge,Malvern"
1,M1C,43.784535,-79.160497,1,Scarborough,"Highland Creek,Rouge Hill,Port Union","M1C, Scarborough, Highland Creek,Rouge Hill,Po..."
2,M1E,43.763573,-79.188711,2,Scarborough,"Guildwood,Morningside,West Hill","M1E, Scarborough, Guildwood,Morningside,West Hill"
3,M1G,43.770992,-79.216917,3,Scarborough,Woburn,"M1G, Scarborough, Woburn"
4,M1H,43.773136,-79.239476,4,Scarborough,Cedarbrae,"M1H, Scarborough, Cedarbrae"


In [161]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 103 entries, 0 to 102
Data columns (total 7 columns):
PostalCode       103 non-null object
Latitude         103 non-null float64
Longitude        103 non-null float64
Unnamed: 0       103 non-null int64
Borough          103 non-null object
Neighbourhood    103 non-null object
address          103 non-null object
dtypes: float64(2), int64(1), object(4)
memory usage: 6.4+ KB


- Rearrange columns and drop foreign key:

In [162]:
cols = data1.columns.tolist()
cols

['PostalCode',
 'Latitude',
 'Longitude',
 'Unnamed: 0',
 'Borough',
 'Neighbourhood',
 'address']

In [163]:
new_column_order = ['PostalCode',
 'Borough',
 'Neighbourhood',
 'Latitude',
 'Longitude']
new_column_order

['PostalCode', 'Borough', 'Neighbourhood', 'Latitude', 'Longitude']

In [164]:
data1 = data1[new_column_order]
#data1.head()

- Sort dataframe to match example:

In [165]:
sorted_df = data1.sort_values([ 'Neighbourhood', 'Latitude'], ascending=[True, True])
#sorted_df.head()
# no idea how to get it exacly like the exqample :(

In [166]:
sorted_df.reset_index(inplace=True)
#sorted_df.head()


In [167]:
sorted_cols =sorted_df.columns.tolist()
#sorted_cols

In [168]:
new_column_order2 = ['PostalCode',
 'Borough',
 'Neighbourhood',
 'Latitude',
 'Longitude']
new_column_order2

['PostalCode', 'Borough', 'Neighbourhood', 'Latitude', 'Longitude']

In [169]:
sorted_dataframe = sorted_df[new_column_order]
sorted_dataframe.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M5H,DowntownToronto,"Adelaide,King,Richmond",43.650571,-79.384568
1,M1S,Scarborough,Agincourt,43.794200,-79.262029
2,M1V,Scarborough,"Agincourt North,L'Amoreaux East,Milliken,Steel...",43.815252,-79.284577
3,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",43.739416,-79.588437
4,M8W,Etobicoke,"Alderwood,Long Branch",43.602414,-79.543484


## 2.6. Submit a link to your Notebook on your Github repository. (2 marks)

In [170]:
sorted_dataframe.to_csv('sorted_geoloc.csv')

#https://github.com/lindangulopez/Coursera_Capstone/blob/master/Q_2latlong.ipynb

# Question 3:


## 3.1. Build a test set with boroughs in Toronto, see:

https://medium.com/@bobhaffner/folium-markerclusters-and-fastmarkerclusters-1e03b01cb7b1



Import dependencies that we will need.

In [171]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


#### many ways to move the data to json, like:

>with open('df.json', 'w', encoding='utf-8') as file:
>    sorted_dataframe.to_json(file, force_ascii=False)

but I'll go with the following to start off :)

In [105]:
# library to handle JSON files

import pandas as pd

import json

sorted_dataframe.to_json(path_or_buf='geo_toronto.json', orient='table')


In [106]:
with open('geo_toronto.json') as json_data:
    Toronto_data = json.load(json_data)

In [108]:
#Toronto_data
# Data is in the 'data' field   

In [109]:
neighborhoods_data = Toronto_data['data']
neighborhoods_data[0]
#Let's take a look at the first item in this list.

{'index': 0,
 'PostalCode': 'M5H',
 'Borough': 'DowntownToronto',
 'Neighbourhood': 'Adelaide,King,Richmond',
 'Latitude': 43.6505712,
 'Longitude': -79.3845675}

In [125]:

sorted_dataframe.info()
sorted_dataframe.shape


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 5 columns):
PostalCode       103 non-null object
Borough          103 non-null object
Neighbourhood    103 non-null object
Latitude         103 non-null float64
Longitude        103 non-null float64
dtypes: float64(2), object(3)
memory usage: 4.1+ KB


(103, 5)

In [126]:
sorted_dataframe.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M5H,DowntownToronto,"Adelaide,King,Richmond",43.650571,-79.384568
1,M1S,Scarborough,Agincourt,43.794200,-79.262029
2,M1V,Scarborough,"Agincourt North,L'Amoreaux East,Milliken,Steel...",43.815252,-79.284577
3,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",43.739416,-79.588437
4,M8W,Etobicoke,"Alderwood,Long Branch",43.602414,-79.543484


In [127]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(sorted_dataframe['Borough'].unique()),
        sorted_dataframe.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


In [128]:
address = 'Adelaide'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Adelaide are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Adelaide are -34.9281805, 138.5999312.


In [129]:
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab


Solving environment: / 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/linux-64::anaconda==5.3.1=py37_0
  - defaults/linux-64::astropy==3.0.4=py37h14c3975_0
  - defaults/linux-64::bkcharts==0.2=py37_0
  - defaults/linux-64::blaze==0.11.3=py37_0
  - defaults/linux-64::bokeh==0.13.0=py37_0
  - defaults/linux-64::bottleneck==1.2.1=py37h035aef0_1
  - defaults/linux-64::dask==0.19.1=py37_0
  - defaults/linux-64::datashape==0.5.4=py37_1
  - defaults/linux-64::mkl-service==1.1.2=py37h90e4bf4_5
  - defaults/linux-64::numba==0.39.0=py37h04863e7_0
  - defaults/linux-64::numexpr==2.6.8=py37hd89afb7_0
  - defaults/linux-64::odo==0.5.1=py37_0
  - defaults/linux-64::pytables==3.4.4=py37ha205bf6_0
  - defaults/linux-64::pytest-arraydiff==0.2=py37h39e3cac_0
  - defaults/linux-64::pytest-astropy==0.4.0=py37_0
  - defaults/linux-64::pytest-doctestplus==0.1.3=py37_0
  - defaults/linux-64::pywavelets==1.0.0=py37h

In [130]:
import pandas as pd
import folium

print('imported pandas & folium')

imported pandas & folium


In [132]:
import pandas as pd
import folium

#grab a random sample from df
subset_of_df = sorted_dataframe.sample(n=50)
map_test = folium.Map(location=[subset_of_df['Latitude'].mean(), 
                                subset_of_df['Longitude'].mean()], 
                      zoom_start=10)
#creating a Marker for each point in df_sample. Each point will get a popup with their zip
for row in subset_of_df.itertuples():
    map_test.add_child(folium.Marker(location=[row.Latitude ,row.Longitude],
           popup=row.Borough))

    
map_test

In [133]:
from folium.plugins import MarkerCluster
map_borough = folium.Map(location=[subset_of_df['Latitude'].mean(), 
 subset_of_df['Longitude'].mean()], 
 zoom_start=10)
mc = MarkerCluster()
#creating a Marker for each point in df_sample. Each point will get a popup with their zip
for row in subset_of_df.itertuples():
    mc.add_child(folium.Marker(location=[row.Latitude,  row.Longitude],
                 popup=row.Borough))

    
map_borough.add_child(mc)


map_borough



### 3.2. Replicate the same analysis with the full data set.

In [135]:
from folium.plugins import MarkerCluster
map_neighbourhood = folium.Map(location=[subset_of_df['Latitude'].mean(), 
 subset_of_df['Longitude'].mean()], 
 zoom_start=10)
mc = MarkerCluster()
#creating a Marker for each point in df_sample. Each point will get a popup with their zip
for row in subset_of_df.itertuples():
    mc.add_child(folium.Marker(location=[row.Latitude,  row.Longitude],
                 popup=row.Neighbourhood))

    
map_neighbourhood.add_child(mc)

map_neighbourhood

## 3.3. Explain what you decided to do and report observations.



## 3.4. Generate maps to visualize your neighborhoods and how they cluster together.



## 3.5. Submit a link to your Notebook on your Github repository. (3 marks)

This notebook is an assignment for a course on **Coursera** called *Applied Data Science Capstone*, you can take this course online by clicking [here](http://cocl.us/DP0701EN_Coursera_Week3_LAB2).